In [36]:
def fas2(adj_list: list[list[int]]) -> tuple[list[list[int]], int]:
    """Remove back edges from a directed graph via depth-first search.

    Args:
        adj_list (list[list[int]]): Adjacency list representing the graph.

    Returns:
        list[list[int]]: Adjacency list of the graph after removing cycles.
    """
    n = len(adj_list)
    color = [0] * n
    removed: set[tuple[int, int]] = set()

    def dfs(u: int):
        """Depth-first search helper that marks back edges for removal.

        Args:
            u (int): Current node being visited.

        Returns:
            None: The function updates the closure state in-place.
        """
        color[u] = 1
        for v in adj_list[u]:
            if color[v] == 0:
                dfs(v)
            elif color[v] == 1:
                removed.add((u, v))
            else:
                pass
        color[u] = 2

    for i in range(n):
        if color[i] == 0:
            dfs(i)

    original_edges = set((u, v) for u in range(n) for v in adj_list[u])
    kept = original_edges - removed

    out_adj: list[list[int]] = [[] for _ in range(n)]
    buckets = [[] for _ in range(n)]
    for u, v in kept:
        buckets[u].append(v)
    for u in range(n):
        out_adj[u] = buckets[u]

    return out_adj, len(removed)

In [37]:
def fas3(adj_list: list[list[int]]) -> tuple[list[list[int]], int]:
    """Remove cycles using an exponential-time dynamic programming approach.

    Args:
        adj_list (list[list[int]]): Adjacency list representing the graph.

    Returns:
        list[list[int]]: Adjacency list of a maximally acyclic subgraph.
    """
    n = len(adj_list)
    if n == 0:
        return [], 0

    # Build, for each v, a bitmask of incoming-edge sources: InMask[v][u]=1 iff u->v exists.
    in_mask = [0] * n
    for u in range(n):
        for v in adj_list[u]:
            if 0 <= v < n and v != u:
                in_mask[v] |= 1 << u

    full = (1 << n) - 1
    # dp[mask] = max number of forward edges achievable using exactly the vertices in 'mask'
    dp = [-(10**18)] * (1 << n)
    best_last = [-1] * (1 << n)  # which vertex is placed last for the optimal dp[mask]
    dp[0] = 0

    # Iterate all non-empty masks
    for mask in range(1, full + 1):
        # iterate v as the last vertex in 'mask'
        m = mask
        while m:
            v_bit = m & -m
            v = v_bit.bit_length() - 1
            prev = mask ^ v_bit
            # if v is last, edges counted are all from 'prev' into v
            gain = (in_mask[v] & prev).bit_count()
            val = dp[prev] + gain
            if val > dp[mask]:
                dp[mask] = val
                best_last[mask] = v
            m ^= v_bit

    # Reconstruct an ordering (from first to last)
    order_rev = []
    cur = full
    while cur:
        v = best_last[cur]
        if v == -1:
            # fallback (shouldn't happen), pick any set bit
            v = (cur & -cur).bit_length() - 1
        order_rev.append(v)
        cur ^= 1 << v
    order = order_rev[::-1]  # now from first to last

    # Keep only edges that go forward under this order
    pos = [-1] * n
    for i, v in enumerate(order):
        pos[v] = i

    dag = [[] for _ in range(n)]
    for u in range(n):
        pu = pos[u]
        if pu == -1:
            continue
        for v in adj_list[u]:
            if 0 <= v < n and pu < pos[v]:
                dag[u].append(v)

    return dag, sum(len(row) for row in adj_list) - sum(len(row) for row in dag)

In [38]:
import random
from typing import List

def generate_sparse_cycle_tournament(n: int, num_flips: int | None = None, seed: int | None = None) -> List[List[int]]:
    """희소 사이클을 가지는 토너먼트 그래프를 생성한다.

    아이디어:
    1. 먼저 0 < 1 < ... < n-1 의 총순서를 따르는 'transitive tournament'를 만든다.
       즉, i < j 이면 i -> j 로 두면 사이클이 전혀 없다.
    2. 그 후, 일부 간선을 골라 방향을 뒤집는다.
       이 '위로 올라가는' 간선들이 사이클을 만들지만, 개수를 적게 유지하면
       전체적으로 사이클이 희소한 토너먼트가 된다.

    Args:
        n: 노드 개수.
        num_flips: 방향을 뒤집을 간선의 개수. None이면 대략 n/2 개 정도만 뒤집는다.
        seed: 난수 시드를 고정하고 싶을 때 사용.

    Returns:
        adj_list: 길이 n 의 리스트. adj_list[u] 는 u 에서 나가는 이웃 v 들의 리스트.
    """
    if n <= 0:
        return []

    if seed is not None:
        random.seed(seed)

    # 1. 완전히 acyclic 한 transitive tournament 생성: i < j 이면 i -> j
    adj = [set() for _ in range(n)]
    forward_edges = []  # (u, v) with u < v, u -> v
    for u in range(n):
        for v in range(u + 1, n):
            adj[u].add(v)
            forward_edges.append((u, v))

    # 2. 일부 간선 방향 뒤집기
    if num_flips is None:
        # "희소"하게: n/2 정도만 뒤집도록 (원하면 더 줄이거나 늘려도 됨)
        num_flips = max(1, n // 3)

    num_flips = min(num_flips, len(forward_edges))
    # 뒤집을 간선들을 무작위로 선택
    flip_candidates = random.sample(forward_edges, k=num_flips)

    for u, v in flip_candidates:
        if v in adj[u]:
            # 기존 u -> v 제거
            adj[u].remove(v)
            # 새로 v -> u 추가
            adj[v].add(u)
        # 만약 이미 누가 건드려서 구조가 바뀌었다면(이론상 거의 없음) 그냥 패스해도 됨

    # set -> list 로 변환
    adj_list: List[List[int]] = [sorted(list(neis)) for neis in adj]
    return adj_list

In [39]:
s = 0
for i in range(100):
    g = generate_sparse_cycle_tournament(10, 5)
    _, a = fas1(g)
    s += a
print(s / 100)

19.4
